In [ ]:
%pip install torch numpy argparse jupyter notebook
%pip install reformer-pytorch

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
   ---------------------------------------- 0.0/13.2 MB ? eta -:--:--
   --------- ------------------------------ 3.1/13.2 MB 15.4 MB/s eta 0:00:01
   ------------------- -------------------- 6.6/13.2 MB 15.5 MB/s eta 0:00:01
   -------------------------- ------------- 8.7/13.2 MB 14.1 MB/s eta 0:00:01
   ------------------------------- -------- 10.5/13.2 MB 11.9 MB/s eta 0:00:01
   ------------------------------------ --- 12.1/13.2 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 13.2/13.2 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.7 MB 5.2 MB/s eta 0:00:03
   -------- ------------------------------- 2.6/11.7 MB 5.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.7 MB 5.1 MB/s eta 0:00:02
   -------------------- ---------

In [ ]:
import os
import torch
import numpy as np
import random
from exp.exp_main import Exp_Main

# Classe Args pour exécuter sans argparse
class Args:
    is_training = 1 # 1: Train & Test & peut prédire
    model_id = 'test' # nom utilisé pour les run
    model = 'Modified_Transformer'  # 
    data = 'ETTh'   # specifie le dataset utilisé
    root_path = './'
    data_path = 'ETTh1.csv'  # Assurez-vous que ce fichier est bien présent
    features = 'M'  # 'M' : predit toutes les colonnes , 'MS' : utilise tte les colonnes et predit un seul outcome
    #                   , 'S' : predit une seule colonne
    target = 'OT'
    freq = 'h' # Fréquence des données (h = heures, d = jours, t = minutes, etc.)

    checkpoints = './checkpoints/'  # Dossier où les modèles entraînés sont sauvegardés

    seq_len = 96
    label_len = 48
    pred_len = 96

    enc_in = 7 # nb de variable en entrée de l'encodeur
    dec_in = 7  # nb de variable en entrée de l'encodeur
    c_out = 7   # nb de classes ??
    d_model = 512
    n_heads = 8
    e_layers = 2 # nb de layers
    d_layers = 1 # nb de decoder
    d_ff = 2048 # dim du FF 
    dropout = 0.1
    activation = 'gelu'
    output_attention = False
    do_predict = True   # generer des pred apres le train
    exp_setting = 0 # 0 : "MLP classique", 2 : "FAN_Gate", 4 : "FAN_wo_gate"
    embed = 'timeF'
    factor = 1
    use_amp = False # precision mixte avec entrainement sur GPU
    lradj = 'type1'  # Méthode d'ajustement du learning rate (ex: 'type1', 'type2', etc.)


    batch_size = 32
    learning_rate = 0.0001
    train_epochs = 2
    patience = 10
    num_workers = 10

    use_gpu = torch.cuda.is_available()
    gpu = 0
    use_multi_gpu = False
    devices = '0'

# Fixe une graine aléatoire
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# Initialise les arguments
args = Args()

# Active le multi-GPU si nécessaire
if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Arguments du modèle :', vars(args))

# Initialise l'expérience
exp = Exp_Main(args)
setting = "test_run"

# Entraînement du modèle
print("Début de l'entraînement...")
exp.train(setting)

# Test du modèle
print("Test du modèle...")
exp.test(setting)

# Prédictions
if args.do_predict:
    print("Prédictions sur de nouvelles données...")
    exp.predict(setting, load=True)

torch.cuda.empty_cache()



Arguments du modèle : {}
Use CPU
the parameters of the model: 10.540039 M
Début de l'entraînement...
train 8449
val 2785
test 2785
	iters: 100, epoch: 1 | loss: 0.4309945
	speed: 7.7950s/iter; left time: 3344.0697s


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Charger les prédictions sauvegardées
preds = np.load("./results/test_run/real_prediction.npy")

# Afficher la forme des données
print("Shape des prédictions :", preds.shape)

# Tracer la première série prédite
plt.figure(figsize=(12, 5))
plt.plot(preds[0, :, 0], label="Prédiction")
plt.legend()
plt.title("Prédictions de Séries Temporelles")
plt.show()
